In [1]:
import uproot
import os,sys

In [2]:
pjoin = os.path.join

In [3]:
## define the directories where input files are present

dataDir = 'inputTreeFiles/'
modelDir= 'nnmodelfiles/'




In [4]:
events = uproot.open(pjoin(dataDir,'VLL_M125_sample.root:events_1L2J'))

In [5]:
events

<TTree 'events_1L2J' (45 branches) at 0x7f309d1cbee0>

In [6]:
## Show all the branch and their type
events.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
index                | int64_t                  | AsDtype('>i8')
nEvt                 | int32_t                  | AsDtype('>i4')
lep0_flavor          | int32_t                  | AsDtype('>i4')
lep0_pt              | float                    | AsDtype('>f4')
lep0_phi             | float                    | AsDtype('>f4')
lep0_eta             | float                    | AsDtype('>f4')
lep0_iso             | float                    | AsDtype('>f4')
lep0_mt              | float                    | AsDtype('>f4')
lep0_sip3d           | float                    | AsDtype('>f4')
lep0_deepjet         | float                    | AsDtype('>f4')
jet0_pt              | float                    | AsDtype('>f4')
jet0_phi             | float                    | AsDtype('>f4')
jet0_eta             | float                    | AsDtype(

In [7]:
# Let's read the tree in a dataframe

df_full =  events.arrays(library='pd')

In [8]:
df_full.head()

,index,nEvt,lep0_flavor,lep0_pt,lep0_phi,lep0_eta,lep0_iso,lep0_mt,lep0_sip3d,lep0_deepjet,...,ljjsys_PT,deltaPhi_ljjsysmet,deltaPhi_ljjsyslep0,deltaPhi_ljjsysjet0,deltaPhi_ljjsysjet1,ljjsys_mass,deepjetQG_jet0,deepjetQG_jet1,event_avgQGscore,event_Rpt
0,0,23,-13,45.887268,1.607178,-0.837402,0.000000,29.566328,2.925781,0.054810,...,148.854080,2.908695,2.525638,0.522701,1.170048,414.921326,0.159424,0.522949,0.341187,0.093759
1,1,29,-13,27.168795,0.016785,-0.995850,0.025366,55.288418,1.150391,0.019623,...,40.035542,1.704648,2.779050,0.657150,0.419004,100.700050,0.904785,0.750000,0.876302,0.582773
2,2,37,-13,66.803604,2.853516,1.022705,0.010576,90.956818,0.644043,0.004219,...,65.152077,2.984196,1.095542,1.701852,0.190347,171.414841,0.313232,0.825684,0.569458,0.113813
3,3,78,13,36.846138,3.106445,1.010498,0.104697,0.673874,0.853027,0.011467,...,40.910255,2.135002,2.150139,0.042246,1.401723,161.187424,0.362793,0.696777,0.529785,0.303758
4,4,105,-13,33.392525,2.098633,-1.825928,0.229012,31.048515,0.982910,0.038177,...,41.693455,2.138128,3.074163,0.136437,2.699744,202.043671,0.748047,0.592773,0.670410,0.183957


In [9]:
#Let's create a dataframe only using variables used for nn training
#We use this dataframe for the evaluation purpose

nnvar = ['lep0_pt','lep0_eta','lep0_mt','jet0_pt','jet0_eta', 'jet0_mt',
         'jet1_pt','jet1_eta', 'jet1_mt','dijet_pt', 'dijet_mt','deltaR_jet01',
         'deltaPhi_metjet0', 'deltaPhi_metjet1', 'deltaPhi_metlep0',
         'deltaPhi_jet0lep0', 'deltaPhi_jet1lep0', 'deltaPhi_dijetlep0','deltaPhi_metdijet',
         'event_MET', 'event_HT', 'n_Jet','deepjetQG_jet0', 'deepjetQG_jet1', 'event_avgQGscore']


df_nn = df_full[nnvar]


### Evaluation of trained NN

In [10]:
import tensorflow as tf
import joblib
import warnings
warnings.filterwarnings('ignore')

In [11]:
modelname  = pjoin(modelDir,'best_model_wjetsvsM125.h5')
scalername = pjoin(modelDir,'scaler_wjetsvsM125.save')
                  
mymodel = tf.keras.models.load_model(modelname)
mymodel.load_weights(modelname)
myscaler = joblib.load(scalername)

2024-01-11 12:41:28.461303: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-11 12:41:28.461696: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-11 12:41:28.463765: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


ValueError: Unknown optimizer: Custom>Adam